In [17]:
from cimaq_utils import *
uzipfiles()
cimaqfilter()
encdf = get_clean_encodings(uzeprimes)
test = [prepsheet(row[1]['fpath'], row[1]['encod'])
        for row in encdf.iterrows()]


100%|██████████| 103/103 [00:01<00:00, 90.64it/s]


Removing empty folder: /home/francois/../../media/francois/seagate_1tb/CIMAQ_fmri_memory/data/task_files/uzeprimes/4509950_108391_Tache_IRM/4509950_108391_Tache_IRM
Removing empty folder: /home/francois/../../media/francois/seagate_1tb/CIMAQ_fmri_memory/data/task_files/uzeprimes/4509950_108391_Tache_IRM
Removing empty folder: /home/francois/../../media/francois/seagate_1tb/CIMAQ_fmri_memory/data/task_files/uzeprimes/4548020_258912_Tache_IRM/4548020_258912_Tache_IRM
Removing empty folder: /home/francois/../../media/francois/seagate_1tb/CIMAQ_fmri_memory/data/task_files/uzeprimes/4548020_258912_Tache_IRM
Removing empty folder: /home/francois/../../media/francois/seagate_1tb/CIMAQ_fmri_memory/data/task_files/uzeprimes/4557550_270218_Tache_IRM/4557550_270218_Tache_IRM
Removing empty folder: /home/francois/../../media/francois/seagate_1tb/CIMAQ_fmri_memory/data/task_files/uzeprimes/4557550_270218_Tache_IRM
Removing empty folder: /home/francois/../../media/francois/seagate_1tb/CIMAQ_fmri_mem

In [18]:
display(test)

[       1  CTL    CTL0   Control     0.025        3
 0      1  CTL    CTL0  Fixation     3.034    1.500
 1      2  Enc   Enc00  Encoding     4.521    3.000
 2      2  Enc   Enc00  Fixation      7.53    0.500
 3      3  Enc  Enc000  Encoding     8.032    3.000
 4      3  Enc  Enc000  Fixation    11.041    0.500
 ..   ...  ...     ...       ...       ...      ...
 234  118  Enc   Enc76     Old18  Fixation  719.936
 235  119  Enc   Enc77     Old33  Encoding  724.934
 236  119  Enc   Enc77     Old33  Fixation  727.943
 237  120  Enc   Enc78     Old14  Encoding  728.929
 238  120  Enc   Enc78     Old14  Fixation  731.938
 
 [239 rows x 6 columns],
     TrialNumber Category TrialCode OldNumber CorrectSource Stim_RESP Stim_RT
 0             1      Enc     Enc00         5             1       793       0
 1             2      Enc    Enc000         9             1       668       0
 2             3      CTL      CTL0         9             1       602       0
 3             4      Enc     Enc01  